In [ ]:
from qiskit.circuit.random import random_circuit
from qiskit import transpile

seed = 42

# Define the number of qubits and the number of gates (depth)
num_qubits = 3
depth = 1

# Generate a random circuit
qc = random_circuit(num_qubits=num_qubits, depth=depth, max_operands=3, measure=False, seed=seed)

# Define the basis set corresponding to the heron QPU
basis_set_heron = ['cx', 'rz', 'ry', 'rx']

# Transpile the circuit to the heron basis set
qc = transpile(qc, basis_gates=basis_set_heron)

qc.global_phase = 0

# Draw the circuit
qc.draw('mpl')

In [ ]:
from qiskit import QuantumCircuit
from math import pi

seed = 42

# Create a quantum circuit with 3 qubits
qc = QuantumCircuit(3)

# Add gates as per the diagram
qc.ry(pi/2, 1)  # RY(pi/2) on q1
qc.rx(pi, 1)    # RX(pi) on q1

qc.cx(2, 1)     # CX with control q1 and target q2

# Add RZ gates
qc.rz(-pi/4, 1)       # RZ(pi/4) on q0
qc.cx(0, 1)          # CX with control q0 and target q1
qc.rz(pi/4, 1)    # RZ(-3pi/4) on q1
qc.cx(2, 1)          # CX with control q0 and target q1
qc.rz(-pi/4, 1)       # RZ(pi/4) on q1
qc.rz(pi/4, 2)       # RZ(pi/4) on q2
qc.cx(0, 1)        # CX with control q0 and target q1
qc.cx(0, 2)        # CX with control q0 and target q2
qc.rz(pi/4, 0)       # RZ(pi/4) on q1
qc.rz(-3*pi/4, 1)       # RZ(pi/4) on q1
qc.rz(-pi/4, 2)       # RZ(pi/4) on q1
qc.ry(pi/2, 1)       # RY(-pi/2) on q1
qc.cx(0, 2)          # CX with control q1 and target q2

qc.draw("mpl")


In [ ]:
# from qiskit import QuantumCircuit
# from qiskit.quantum_info import random_unitary
# import numpy as np

# seed = 42

# # Create a Quantum Circuit with 2 qubits
# qc = QuantumCircuit(10)

# qc.unitary(random_unitary(2**10), range(10))
# qc.unitary(random_unitary(2**9) , range(9) )

# # Draw the circuit
# qc.draw('mpl')

In [ ]:
# from qiskit import QuantumCircuit
# from qiskit.quantum_info import random_unitary
# import numpy as np

# seed = 42

# # Create a Quantum Circuit with 2 qubits
# qc = QuantumCircuit(3)

# qc.unitary(random_unitary(2**3), [0, 1, 2])
# qc.unitary(random_unitary(2**2), [0, 1])

# # Draw the circuit
# qc.draw('mpl')

In [ ]:
# from qiskit import QuantumCircuit
# from qiskit.quantum_info import random_unitary
# import numpy as np

# seed = 42
# np.random.seed(seed)

# def pick_random_qubits(num_qubits, seed):
#     # choose a random number of qubits from 1 to num_qubits
#     num_qubits_chosen = np.random.randint(1, num_qubits + 1)
#     # choose a random set of qubits
#     qubits = np.random.choice(num_qubits, num_qubits_chosen, replace=False)
#     qubits = sorted([int(q) for q in qubits])
#     return qubits, num_qubits_chosen

# def create_random_unitary(num_qubits_chosen, seed):
#     return random_unitary(2**num_qubits_chosen, seed=seed)


# number_of_gates = 100
# n_qubits_in_circuit = 10

# # Create a Quantum Circuit with 2 qubits
# qc = QuantumCircuit(n_qubits_in_circuit)

# for _ in range(number_of_gates):
#     qubits_chosen, num_qubits_chosen = pick_random_qubits(n_qubits_in_circuit, seed)
#     qc.unitary(create_random_unitary(num_qubits_chosen, seed), qubits_chosen)


# # Draw the circuit
# qc.draw('mpl')

In [ ]:
class Gate:
    def __init__(self, qubits, unitary):
        self.qubits = qubits  # span of qubits on which the gate acts
        self.unitary = unitary  # unitary matrix representing the gate

    def __repr__(self):
        return f'Gate(qubits={self.qubits}, unitary=\n{self.unitary})'

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator

def remap_qubits(qubits):
    sorted_unique = sorted(set(qubits))
    rank_dict = {num: rank for rank, num in enumerate(sorted_unique)}
    return [rank_dict[num] for num in qubits]

gates_list = []

n_qubits = qc.num_qubits

for instruction in qc.data:
    gate_operation = instruction.operation
    # qubits = [n_qubits - qubit._index - 1 for qubit in instruction.qubits]
    qubits = [qubit._index for qubit in instruction.qubits]
    
    # single_gate_circuit = QuantumCircuit(len(qubits))
    # single_gate_circuit.append(gate_operation, remap_qubits(qubits))
    # unitary_matrix = Operator(single_gate_circuit).data

    # gates_list.append(Gate(qubits, unitary_matrix))
    gates_list.append(Gate(qubits, instruction.matrix))

for gate in gates_list:
    print(gate)
    print()

In [ ]:
class TreeNode:
    def __init__(self, gate = None):
        self.left = None
        self.right = None
        self.span = None if gate is None else gate.qubits
        self.gate = gate

    def __repr__(self):
        return f'TreeNode(span={self.span}, left={self.left}, right={self.right})'

In [ ]:
def build_tree(node_list):
    # iterate the list, when you find two consecutive gates having some qubits in common, create a new node with these gates as children and remove them from the list

    while len(node_list) > 1:
        for i in range(len(node_list) - 1):
            if node_list[i].span is None or node_list[i + 1].span is None:
                continue

            if len(set(node_list[i].span).intersection(node_list[i + 1].span)) > 0:
                new_node = TreeNode()
                new_node.left  = node_list[i]
                new_node.right = node_list[i + 1]
                new_node.span = list(set(node_list[i].span).union(node_list[i + 1].span))
                node_list[i] = new_node
                node_list.pop(i + 1)
                break

    return node_list[0]

In [ ]:
import time

start_time = time.time()

node_list = [TreeNode(gate) for gate in gates_list]
root = build_tree(node_list)

end_time = time.time()
contraction_time = end_time - start_time

print(root)
print(len(node_list))
print(f"Contraction time: {contraction_time} seconds")

In [ ]:
# create four sqlite tables: gates, qubits, circuit, circuit_gates and a db
import sqlite3

db_filename = 'data/quantum_circuit.db'

# remove the db
import os
try:
    os.remove(db_filename)
except FileNotFoundError:
    passgate_operation

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect(db_filename)
c = conn.cursor()

# Create additional tables
c.execute('''CREATE TABLE IF NOT EXISTS contractions (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                program_id INTEGER NOT NULL,
                span TEXT NOT NULL,
                left_id INTEGER DEFAULT NULL,
                right_id INTEGER DEFAULT NULL,
                kind TEXT NOT NULL,
                gate_id INTEGER DEFAULT NULL,
                FOREIGN KEY (program_id) REFERENCES programs (id),
                FOREIGN KEY (left_id) REFERENCES contractions (id),
                FOREIGN KEY (right_id) REFERENCES contractions (id),
                CHECK( ( kind = 'C' AND left_id IS NOT NULL AND right_id IS NOT NULL AND gate_id IS NULL ) 
                       OR ( kind = 'G' AND left_id IS NULL AND right_id IS NULL AND gate_id IS NOT NULL ) )
            )''')

c.execute('''CREATE TABLE IF NOT EXISTS experiments (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                program_id INTEGER NOT NULL,
                input_vector BLOB NOT NULL,
                output_vector BLOB NOT NULL,
                FOREIGN KEY (program_id) REFERENCES programs (id)
            )''')

c.execute('''CREATE TABLE IF NOT EXISTS gates (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                rank INTEGER NOT NULL,
                data BLOB NOT NULL
            )''')

c.execute('''CREATE TABLE IF NOT EXISTS programs (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                filename TEXT NOT NULL,
                contraction_cpu_time_us INTEGER DEFAULT NULL,
                tree_building_time_us INTEGER DEFAULT NULL
            )''')

# insert a program into the db
filename = 'test.qasm'
text = "NULL"
contraction_cpu_time_us = int(contraction_time * 1e6)

c.execute('''INSERT INTO programs (filename, contraction_cpu_time_us) VALUES (?, ?)''', (filename, contraction_cpu_time_us))

# insert the tree into the db
def insert_tree(node):
    if node.span is None:
        return

    if node.left is None and node.right is None:
        kind = 'G'
        # add gate to the gates table
        c.execute('''INSERT INTO gates (rank, data) VALUES (?, ?)''', (len(node.gate.qubits), node.gate.unitary.tobytes()))
        gate_id = c.lastrowid
        # min_span = min(node.span)
        # max_span = max(node.span)
        # node_span_str = f'[{min_span}, {max_span}]'
        node_span_str = str(node.span)
        c.execute('''INSERT INTO contractions (program_id, span, kind, gate_id) VALUES (?, ?, ?, ?)''', (1, node_span_str, kind, gate_id))
    else:
        kind = 'C'
        left_id = insert_tree(node.left)
        right_id = insert_tree(node.right)
        c.execute('''INSERT INTO contractions (program_id, span, kind, left_id, right_id) VALUES (?, ?, ?, ?, ?)''', (1, str(node.span), kind, left_id, right_id))

    return c.lastrowid

insert_tree(root)

# Commit changes and close the connection
conn.commit()
conn.close()


In [ ]:
import numpy as np
import time
from qiskit_aer import Aer
from qiskit.quantum_info import random_statevector

# Generate a random state vector
input_state = random_statevector(2**qc.num_qubits, seed=seed)

# Set up the backend
backend = Aer.get_backend('statevector_simulator')

qc_init = QuantumCircuit(qc.num_qubits)

# Initialize the input state
qc_init.initialize(input_state.data, range(qc.num_qubits))

# Combine the initialization circuit with the original circuit
qc_init.compose(qc, inplace=True)

# Execute the circuit
start_time = time.time()
result = backend.run(qc_init).result()
end_time = time.time()

# Get the output state vector
output_state = result.get_statevector(qc_init)

result_referece = np.asarray(output_state, np.complex128)

# Calculate the time taken
execution_time = end_time - start_time

print(f"Input state: {input_state}")
print(f"Output state: {output_state}")
print(f"Execution time: {execution_time} seconds")

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect(db_filename)
c = conn.cursor()

# insert an experiment into the db
input_vector  = np.asarray(input_state,  np.complex128).data.tobytes()
output_vector = np.asarray(output_state, np.complex128).data.tobytes()

c.execute('''INSERT INTO experiments (program_id, input_vector, output_vector) VALUES (?, ?, ?)''', (1, input_vector, output_vector))

# Commit changes and close the connection
conn.commit()
conn.close()

In [ ]:
from qiskit.quantum_info import Operator

# Get the unitary matrix of the quantum circuit
unitary_matrix = Operator(qc).data

print("Unitary matrix of the circuit:")
print(unitary_matrix)

In [ ]:
from qiskit import QuantumCircuit, transpile    
from qiskit_aer import AerSimulator


simulator = AerSimulator(method='unitary', device='GPU')
start_time = time.time()
qc.save_unitary()
transpiled = transpile(qc, simulator)
job = simulator.run(transpiled)
result = job.result()

end_time = time.time()
unitary_matrix = result.get_unitary(transpiled)

execution_time_ms = (end_time - start_time) * 1000

print(f"Execution time: {execution_time_ms} milliseconds")
print(unitary_matrix)

In [ ]:
import numpy as np

result_dot = np.dot(unitary_matrix, input_state.data)

print("Output state vector obtained by multiplying the unitary matrix with the input state vector:")
print(result_dot)

print("Reference output state vector:")
print(result_referece)

In [ ]:
print(unitary_matrix)

In [ ]:
first_gate = gates_list[0]
second_gate = gates_list[1]

a = first_gate.unitary[0, 0]
b = first_gate.unitary[2, 0]
s = second_gate.unitary[0, 0]
t = second_gate.unitary[0, 2]

print(f"Multiplying {a} * {s} + {b} * {t}")

res = a * s + b * t

print(f"Result: {res}")